In [18]:
import rethinkdb as r
from datetime import datetime
import json

r.connect(host="localhost", db="yelp", port=28015).repl()
####################################
# GET TOP REVIEWED BUSINESSES
top_reviewed_businesses = r.table("businesses").order_by(
    r.desc('review_count')
).pluck('name', 'business_id', 'review_count', 'stars').limit(10).run()
########################################################

####################################
# GET TOP REVIEWED TIPS
tips = r.table("tips").eq_join('business_id', r.table('businesses')).zip().group('business_id')\
    .count().run()

tips = sorted(tips.items(), key=lambda x: x[1], reverse=True)

def getBusinesses(a):
    business = r.table("businesses").get(a[0]).pluck('name', 'business_id', 'review_count', 'stars').run()
    business["total_tips"] = a[1]
    return business

top_tipped_businesses = list(map(lambda a: getBusinesses(a), tips[0:10]))
#############################################

####################################
# GET SET OF TOP REVIEW BUSINESS NAMES
top_reviewed_businesses_set = set()
for business in top_reviewed_businesses:
    top_reviewed_businesses_set.add(business["business_id"])
####################################

####################################
# GET SET OF TOP TIP BUSINESS NAMES
top_tipped_businesses_set = set()
for business in top_tipped_businesses:
    top_tipped_businesses_set.add(business["business_id"])
####################################

####################################
# CREATE DICT OF TOP REVIEWS AND TIPS
review_dict = {}
tip_dict = {}

reviews = r.table("reviews").run()
for review in reviews:
    business_id = review["business_id"]
    if business_id in top_reviewed_businesses_set:
        if business_id in review_dict:
            review_dict[business_id].append(review)
        else:
            review_dict[business_id] = [review]

tips = r.table("tips").run()
for tip in tips:
    business_id = tip["business_id"]
    if business_id in top_tipped_businesses_set:
        if business_id in tip_dict:
            tip_dict[business_id].append(tip)
        else:
            tip_dict[business_id] = [tip]
####################################




In [21]:
top_tipped_places = []
top_reviewed_places = []

def sortText(x):
    return datetime(*map(int, x["date"].split("-")))

for business in top_reviewed_businesses:
    top_reviewed_places.append({'name': business['name'], 'reviews': sorted(review_dict[business["business_id"]], key=sortText)})

for business in top_tipped_businesses:
    top_tipped_places.append({'name': business['name'], 'tips': sorted(tip_dict[business["business_id"]], key=sortText)})


In [22]:
reviews = {'places': top_reviewed_places}
tips = {'places': top_tipped_places}
with open('top_reviewed.json', 'w') as outfile:
    json.dump(reviews, outfile)
with open('top_tipped.json', 'w') as outfile:
    json.dump(tips, outfile)